<a href="https://colab.research.google.com/github/noetarbouriech/is-it-gorafi/blob/main/is_it_gorafi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Is it Gorafi?

Text classification model for recognizing Gorafi news articles by comparing with Figaro news article.

## Dependencies installation

In [ ]:
!git clone https://github.com/noetarbouriech/is-it-gorafi.git

Cloning into 'is-it-gorafi'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 3), reused 6 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 59.05 KiB | 491.00 KiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
!pip install datasets transformers huggingface_hub evaluate scikit-learn optimum[exporters]
!apt-get install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Model training

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, TrainingArguments, Trainer

SEED=42

# Load dataset
dataset = load_dataset("csv", data_files="is-it-gorafi/dataset.csv")
dataset = dataset.map(lambda x: {"is_gorafi": int(x["is_gorafi"])})

# Split train/test data
train_test_split = dataset["train"].train_test_split(test_size=0.2, seed=SEED, shuffle=True)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("Geotrend/distilbert-base-en-fr-cased")
model = AutoModelForSequenceClassification.from_pretrained("Geotrend/distilbert-base-en-fr-cased")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["title"], padding="max_length", truncation=True)

# Tokenize dataset
tokenized_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_test_datasets = test_dataset.map(tokenize_function, batched=True)

# Rename label column
tokenized_datasets = tokenized_datasets.rename_column("is_gorafi", "label")
tokenized_test_datasets = tokenized_test_datasets.rename_column("is_gorafi", "label")

# Training arguments
training_args = TrainingArguments(
    output_dir="test_trainer",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=20,
    learning_rate=2e-5,  # Reduced learning rate
    weight_decay=0.1,  # L2 regularization
    per_device_train_batch_size=32,  # Increase batch size
    #load_best_model_at_end=True,  # Load best model based on validation metrics
    greater_is_better=True,  # If True, selects the model with highest accuracy
    report_to="none",  # Disable logging to Weights & Biases
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_test_datasets
)

# Train model
trainer.train()
trainer.evaluate()

# Initialize the sentiment analysis pipeline
nlp_ara = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Process dataset
def analyze_sentiment(batch):
    titles = batch['title']
    predictions = nlp_ara(titles)  # Run sentiment analysis on the batch of titles

    # Prepare the output
    predicted_is_gorafi = [1 if prediction['label'] == 'LABEL_1' else 0 for prediction in predictions]

    # Return the batch with the added 'predicted_is_gorafi' field
    return {'predicted_is_gorafi': predicted_is_gorafi}

# Apply the sentiment analysis function to the dataset
test_dataset = test_dataset.map(analyze_sentiment, batched=True)

nb_examples = min(100, len(test_dataset))
accuracy = 0

# Show some results - accessing the first few entries correctly
for i in range(nb_examples):  # Show the first 5 examples
    example = test_dataset[i]  # Access each example
    print(f"Title: {example['title']}")
    print(f"Predicted is_gorafi: {example['predicted_is_gorafi']} | Actual is_gorafi: {example['is_gorafi']}")
    if example['predicted_is_gorafi'] == example['is_gorafi']:
      accuracy+=1

print("Accuracy = ", accuracy / nb_examples)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at Geotrend/distilbert-base-en-fr-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,0.411489
2,No log,0.350016
3,No log,0.354100
4,No log,0.406253
5,No log,0.543961
6,No log,0.542046
7,No log,0.604932


Epoch,Training Loss,Validation Loss
1,No log,0.411489
2,No log,0.350016
3,No log,0.354100
4,No log,0.406253
5,No log,0.543961
6,No log,0.542046
7,No log,0.604932
8,No log,0.736801
9,No log,0.685950
10,No log,0.694836


Device set to use cuda:0


Map:   0%|          | 0/318 [00:00<?, ? examples/s]

Title: Acheté dans une brocante au Mans en 2010, le tableau s’avère être un Modigliani
Predicted is_gorafi: 0 | Actual is_gorafi: False
Title: Un «film coup de poing»:Le Mohicanmet tous les maux de la Corse à l’écran
Predicted is_gorafi: 0 | Actual is_gorafi: False
Title: Jean-Pierre Pernaut sera inhumé dans un cercueil fabriqué à la main par le dernier petit artisan du Poitou
Predicted is_gorafi: 1 | Actual is_gorafi: True
Title: Les cérémonies des Jeux olympiques récompensées, une Victoire de la musique hors-norme
Predicted is_gorafi: 0 | Actual is_gorafi: False
Title: Présidentielle – Un premier sondage place Cthulhu au second tour
Predicted is_gorafi: 1 | Actual is_gorafi: True
Title: Législatives – N’Golo Kanté toujours en tête des intentions de vote
Predicted is_gorafi: 1 | Actual is_gorafi: True
Title: Expulsions – Le RN remercie Valérie Pécresse d’avoir relié Châtelet à Orly en seulement 25 minutes
Predicted is_gorafi: 1 | Actual is_gorafi: True
Title: Budget : «Je voterai la c

## Using our model

### Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

# Extract true labels and predictions
true_labels = [example['is_gorafi'] for example in test_dataset]
predicted_labels = [example['predicted_is_gorafi'] for example in test_dataset]

# Compute confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[257  37]
 [ 52 289]]


### Using model with a sentence provided by us

In [ ]:
test_title = "François Durovray favorable à des «peines planchers» en cas d'attaque envers les forces de l'ordre"
prediction = nlp_ara(test_title)

# Check confidence score and adjust prediction logic
label = prediction[0]['label']
score = prediction[0]['score']

if score < 0.6:  # Set a threshold for confidence score
    predicted_is_gorafi = 0  # If confidence is low, treat it as not is_gorafi
else:
    predicted_is_gorafi = 1 if label == 'LABEL_1' else 0

print(f"Title: {test_title}")
print(f"Predicted is_gorafi: {predicted_is_gorafi} with confidence score: {score}")

Title: François Durovray favorable à des «peines planchers» en cas d'attaque envers les forces de l'ordre
Predicted is_gorafi: 0 with confidence score: 0.9889334440231323


## Exporting our tokenizer and model in ONNX format

In [ ]:
from optimum.exporters.onnx import main_export

onnx_output_dir = "onnx_model"

main_export(
    model_name_or_path="test_trainer/checkpoint-357",
    task="text-classification",
    output=onnx_output_dir,
)
print(f"ONNX model exported to {onnx_output_dir}/ folder")


ONNX model exported to onnx_model/
